# Virtual screening

Ligand-based virtual screening (LBVS) involves predicting bioactivity of large collections of compounds. It involves multiple steps, which are quite easy to perform with scikit-fingerprints.

Notebook inspired by [Tutorial for the Teach-Discover-Treat (TDT) Competition 2014](https://github.com/sriniker/TDT-tutorial-2014/tree/master) by Sereina Riniker and Gregory Landrum.

First, we will load the Malaria HTS dataset from the above competition. It was created in a series of high-throughput screening (HTS) campaigns from around 2010-2012. It contains an already binarized bioactivity labels.

For shorter calculations in this tutorial, we will subsample the negative class to 100 thousand samples.

In [1]:
import pandas as pd
import numpy as np

df = pd.read_parquet("../data/malaria_hts_train.parquet")

# remove ambiguous labels
df = df[df["label"] != "ambiguous"]

# get binary integer labels
df["label"] = df["label"].map({"false": 0, "true": 1})

smiles_list = df["SMILES"].values
labels = df["label"].values

# subsample
pos_idx = np.where(labels == 1)[0]
neg_idx = np.random.choice(np.where(labels == 0)[0], size=100_000, replace=False)
idx = np.concatenate([pos_idx, neg_idx])
smiles_list = smiles_list[idx].tolist()
labels = labels[idx]

print(f"Positive samples: {len(pos_idx)}")
print(f"Negative samples: {len(neg_idx)}")
print(f"Shapes: SMILES list {len(smiles_list)}, labels {len(labels)}")

Positive samples: 1528
Negative samples: 100000
Shapes: SMILES list 101528, labels 101528


**Exercise 1**

This dataset contains some invalid molecules, e.g. breaking valence rules in modern RDKit. Use `MolFromSmilesTransformer`, and note that:
- some molecules may error out, so use `valid_only` and `.transform_x_y()`
- you can parallelize it with `n_jobs` and `batch_size` for efficiency, just like fingerprints.

Use [documentation](https://scikit-fingerprints.readthedocs.io/latest/api_reference.html) if necessary.

In [2]:
from skfp.preprocessing import MolFromSmilesTransformer

mol_from_smiles = MolFromSmilesTransformer(
    valid_only=True, n_jobs=-1, batch_size=1000, verbose=True
)

mols, labels = mol_from_smiles.transform_x_y(smiles_list, labels)

print(f"Removed molecules: {len(smiles_list) - len(mols)}")

  0%|          | 0/101 [00:00<?, ?it/s]

Removed molecules: 1


## Molecular filters

Typically, in LBVS we have huge collections of molecules. Many of those are highly reactive, promiscuous compounds, dyes, or contain toxic functional groups. Thus, we use **molecular filters** to remove them. They are sets of rules to remove unwanted compounds.

They can be divided into 2 groups:

1. **Physicochemical filters** - define allowed ranges of physicochemical descriptors (e.g. molecular weight, logP, HBA, HBD) and keep only molecules fulfilling those requirements. Examples include [Lipinski's rule of 5](https://scikit-fingerprints.readthedocs.io/latest/modules/generated/skfp.filters.LipinskiFilter.html#skfp.filters.LipinskiFilter) and [REOS](https://scikit-fingerprints.readthedocs.io/latest/modules/generated/skfp.filters.REOSFilter.html#skfp.filters.REOSFilter).
2. **Substructural filters** - define unwanted substructures with SMARTS and keep only molecules that do not contain those patterns. Examples include [PAINS](https://scikit-fingerprints.readthedocs.io/latest/modules/generated/skfp.filters.PAINSFilter.html) and [Brenk filter](https://scikit-fingerprints.readthedocs.io/latest/modules/generated/skfp.filters.BrenkFilter.html).

RDKit contains 10 substructural filters, but no physicochemical filters. Scikit-fingerprints implements 31 filters, 21 physicochemical and 10 substructural. API is based on [feature-engine](https://feature-engine.trainindata.com/en/latest/) and other scikit-learn compatible libraries for preprocessing data.

Let's see an example for a substructural filter with RDKit.

In [3]:
from rdkit.Chem import FilterCatalog
from rdkit.Chem.rdfiltercatalog import FilterCatalogParams
from tqdm.notebook import tqdm

filter_catalog_pains_a = FilterCatalogParams.FilterCatalogs.PAINS_A
params = FilterCatalog.FilterCatalogParams()
params.AddCatalog(filter_catalog_pains_a)
filters = FilterCatalog.FilterCatalog(params)

mols_filtered_rdkit = []
labels_filtered_rdkit = []
for mol, label in tqdm(zip(mols, labels), total=len(mols)):
    matches = filters.GetMatches(mol)
    if not matches:
        mols_filtered_rdkit.append(mol)
        labels_filtered_rdkit.append(label)

<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for class boost::shared_ptr<class RDKit::FilterHierarchyMatcher> already registered; second conversion method ignored.
<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for class boost::shared_ptr<class RDKit::FilterCatalogEntry> already registered; second conversion method ignored.


  0%|          | 0/101527 [00:00<?, ?it/s]

And with scikit-fingerprints:

In [4]:
from skfp.filters import PAINSFilter


filter_pains = PAINSFilter(variant="A", n_jobs=-1, verbose=True)

mols_filtered_skfp, labels_filtered_skfp = filter_pains.transform_x_y(mols, labels)

  0%|          | 0/24 [00:00<?, ?it/s]

### Exercise 2

Lipinski's rule of 5 is arguably the first and most famous molecular filters. It is a physicochemical filter, where a molecule can violate at most one of the rules:

- molecular weight $\leq 500$
- hydrogen bond acceptors (HBA) $\leq 10$
- hydrogen bond donors (HBD) $\leq 5$
- logP $\leq 5$

Implement the Lipinski's rule using RDKit and `for` loop, and filter molecules. Remember to also filter labels for consistency! Relevant RDKit descriptors are in [rdkit.Chem.Descriptors](https://www.rdkit.org/docs/source/rdkit.Chem.Descriptors.html), [rdkit.Chem.rdMolDescriptors](https://www.rdkit.org/docs/source/rdkit.Chem.rdMolDescriptors.html), and [rdkit.Chem.Crippen](https://www.rdkit.org/docs/source/rdkit.Chem.Crippen.html) modules.

Then, use the [scikit-fingerprints LipinskiFilter](https://scikit-fingerprints.readthedocs.io/latest/modules/generated/skfp.filters.LipinskiFilter.html#skfp.filters.LipinskiFilter) and compare the two.

In [5]:
from rdkit.Chem.Crippen import MolLogP
from rdkit.Chem.Descriptors import MolWt
from rdkit.Chem.rdMolDescriptors import CalcNumLipinskiHBA, CalcNumLipinskiHBD


mols_filtered_rdkit = []
labels_filtered_rdkit = []
for mol, label in tqdm(zip(mols, labels), total=len(mols)):
    rules = [
        MolWt(mol) <= 500,  # molecular weight
        CalcNumLipinskiHBA(mol) <= 10,  # HBA
        CalcNumLipinskiHBD(mol) <= 5,  # HBD
        MolLogP(mol) <= 5,  # logP
    ]

    if sum(rules) >= 3:
        mols_filtered_rdkit.append(mol)
        labels_filtered_rdkit.append(label)

  0%|          | 0/101527 [00:00<?, ?it/s]

In [6]:
from skfp.filters import LipinskiFilter

lipinski_filter = LipinskiFilter(n_jobs=-1, verbose=True)
mols_filtered_skfp, labels_filtered_skfp = lipinski_filter.transform_x_y(mols, labels)

  0%|          | 0/24 [00:00<?, ?it/s]

**Exercise 3**

Scikit-learn `Pipeline` does not support `.transform_x_y()`, so in order to use multiple filters, we need to apply them one after another. A common example is using multiple PAINS sets A, B, C.

Create a list of filters `PAINSFilter` with different variants A, B and C. Apply filtering on molecules and labels with all three filters one after another.

Save final results in variables `mols_filtered` and `labels_filtered`, which we will use in the rest of the notebook.

In [7]:
from skfp.filters import PAINSFilter

pains_filters = [
    PAINSFilter(variant="A", n_jobs=-1),
    PAINSFilter(variant="B", n_jobs=-1),
    PAINSFilter(variant="C", n_jobs=-1),
]

mols_filtered = mols
labels_filtered = labels

for pains_filter_variant in pains_filters:
    mols_filtered, labels_filtered = pains_filter_variant.transform_x_y(
        mols_filtered, labels_filtered
    )

Let's check how many molecules we have after filtering:

In [8]:
n_active = labels_filtered.sum()
n_inactive = len(labels_filtered) - n_active

print(f"Number of molecules: before {len(mols)}, after {len(mols_filtered)}")
print(f"Inactive molecules: {n_inactive}")
print(f"Active molecules: {n_active}")

Number of molecules: before 101527, after 94308
Inactive molecules: 93093
Active molecules: 1215


## Data splits

In previous notebook, we used predefined splits. For new datasets, we need to compute them explicitly. Doing this with RDKit or DeepChem requires quite a lot of boilerplate code. Let's see the scaffold split as the most popular example, computed with RDKit. Here, the test set consists of the least frequent scaffolds.

In [9]:
from copy import deepcopy
from rdkit.Chem.Scaffolds import MurckoScaffold
from rdkit import Chem
from collections import defaultdict
from math import ceil

data_size = len(mols)
test_size = ceil(0.2 * data_size)
train_size = data_size - test_size

scaffold_sets = defaultdict(list)
for idx, mol in enumerate(mols):
    mol = deepcopy(mol)
    Chem.RemoveStereochemistry(mol)
    scaffold = MurckoScaffold.MurckoScaffoldSmiles(mol=mol)
    scaffold_sets[scaffold].append(idx)

# sort ascending by size of scaffold set
scaffold_sets = list(scaffold_sets.values())
scaffold_sets.sort(key=len)

# assign the least frequent sets to test, rest to train
train_idxs: list[int] = []
test_idxs: list[int] = []
for scaffold_set in scaffold_sets:
    if len(test_idxs) < test_size:
        test_idxs.extend(scaffold_set)
    else:
        train_idxs.extend(scaffold_set)

mols_train = [mols[i] for i in train_idxs]
mols_test = [mols[i] for i in test_idxs]

y_train = labels[train_idxs]
y_test = labels[test_idxs]

In [10]:
len(mols_train), len(mols_test)

(81218, 20309)

**Exercise 4**

Scikit-fingerprints implements [5 data splitting functions](https://scikit-fingerprints.readthedocs.io/latest/modules/model_selection.html), including train-test and train-valid-test splitters.

Using scikit-fingerprints, implement scaffold splitting. Remember to set `random_state=0` for MaxMin for deterministic results.

Use variable names like `mols_train_scaffold`, `mols_test_scaffold`, etc.

In [11]:
from skfp.model_selection import scaffold_train_test_split


mols_train_scaffold, mols_test_scaffold, y_train_scaffold, y_test_scaffold = (
    scaffold_train_test_split(mols, labels, test_size=0.2)
)

In [12]:
len(mols_train_scaffold), len(mols_test_scaffold)

(81218, 20309)

## Similarity searching LBVS

In similarity searching ligand-based virtual screening, the algorithm looks as follows:

1. Split into train and test
2. Keep only actives (class 1) from train
3. Vectorize all molecules, e.g. with molecular fingerprints
4. For a new molecule from test, calculate similarity values to all train actives. Aggregate them, e.g. as maximum similarity (max fusion)
5. Sort test molecules from highest to lowest score

This way, we have a sorted list of new molecules, from most to least promising. We are interested in top molecules, e.g. best 100, to synthesize and check in the lab.

Let's see how this works with RDKit.

In [13]:
from rdkit.ML.Scoring.Scoring import CalcBEDROC
from rdkit.DataStructs import TanimotoSimilarity
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator

y_train_active_mask = y_train_scaffold == 1
mols_active_train_scaffold = np.array(mols_train_scaffold)[y_train_active_mask]

gen = GetMorganGenerator(radius=2, fpSize=2048)
X_train = [gen.GetFingerprint(mol) for mol in mols_active_train_scaffold]
X_test = [gen.GetFingerprint(mol) for mol in mols_test]

similarities = []
for test_mol in tqdm(X_test):
    similarities.append([])
    for train_pos_mol in X_train:
        sim = TanimotoSimilarity(test_mol, train_pos_mol)
        similarities[-1].append(sim)

similarities = np.array(similarities)
similarities = similarities.max(axis=-1)

sim_sort_idx = np.argsort(similarities)[::-1]

similarities_sorted = similarities[sim_sort_idx]
y_test_sorted = y_test_scaffold[sim_sort_idx]

scores = list(zip(similarities_sorted, y_test_sorted))

bedroc = CalcBEDROC(scores, col=1, alpha=20)
print(f"BEDROC: {bedroc:.2%}")

  0%|          | 0/20309 [00:00<?, ?it/s]

BEDROC: 32.50%


With scikit-fingerprints, this becomes shorter. This is quite similar to kNN from previous notebook, but here we use the `bulk_` functions. They are optimized with Numba to accelerate bulk computation of one-to-many similarities. This way, Python is basically as fast as C++.

We also implemented metrics as functions with scikit-learn interface. This way, you can also import everything from a single place.

In [14]:
from skfp.distances import bulk_tanimoto_binary_similarity
from skfp.metrics import bedroc_score

from skfp.fingerprints import ECFPFingerprint

y_train_active_mask = y_train_scaffold == 1
mols_active_train_scaffold = np.array(mols_train_scaffold)[y_train_active_mask]

fp = ECFPFingerprint(n_jobs=-1)
X_train = fp.transform(mols_active_train_scaffold)
X_test = fp.transform(mols_test)

sims = bulk_tanimoto_binary_similarity(X_train, X_test)
y_pred_scores = np.max(sims, axis=0)

bedroc = bedroc_score(y_test, y_pred_scores)
print(f"BEDROC: {bedroc:.2%}")

BEDROC: 32.57%


**Exercise 5**

Implement a variant of the method above, using:
- [randomized scaffold split](https://scikit-fingerprints.readthedocs.io/latest/modules/generated/skfp.model_selection.randomized_scaffold_train_test_split.html#skfp.model_selection.randomized_scaffold_train_test_split) with 10 different seeds
- ECFP count fingerprint with [Tanimoto count variant](https://scikit-fingerprints.readthedocs.io/latest/modules/generated/skfp.distances.bulk_tanimoto_count_similarity.html#skfp.distances.bulk_tanimoto_count_similarity)
- report average and standard deviation of BEDROC and [enrichment factor at 5%](https://scikit-fingerprints.readthedocs.io/latest/modules/generated/skfp.metrics.enrichment_factor.html) across those 10 runs

In [15]:
from skfp.model_selection import randomized_scaffold_train_test_split
from skfp.metrics import enrichment_factor
from tqdm.notebook import tqdm


bedroc_values = []
ef_values = []
for random_state in tqdm(list(range(10))):
    mols_train, mols_test, y_train, y_test = randomized_scaffold_train_test_split(
        mols, labels, test_size=0.2, random_state=random_state
    )

    y_train_active_mask = y_train == 1
    mols_active_train = np.array(mols_train)[y_train_active_mask]

    fp = ECFPFingerprint(count=True, n_jobs=-1)
    X_train = fp.transform(mols_active_train_scaffold)
    X_test = fp.transform(mols_test)

    sims = bulk_tanimoto_binary_similarity(X_train, X_test)
    y_pred_scores = np.max(sims, axis=0)

    bedroc = bedroc_score(y_test, y_pred_scores)
    ef = enrichment_factor(y_test, y_pred_scores)

    bedroc_values.append(bedroc)
    ef_values.append(ef)


bedroc_mean = np.mean(bedroc_values)
bedroc_std = np.std(bedroc_values)

ef_mean = np.mean(ef_values)
ef_std = np.std(ef_values)

print(f"BEDROC: {bedroc_mean:.2%} +- {bedroc_std:.2%}")
print(f"EF 5%: {ef_mean:.3f} +- {ef_std:.3f}")

  0%|          | 0/10 [00:00<?, ?it/s]

BEDROC: 84.29% +- 2.20%
EF 5%: 16.496 +- 0.428
